In [2]:
import pandas as pd
d = {"headers": {"Employee": ["id", "name", "department", "managerId"]}, "rows": {"Employee": [[101, "John", "A", None],[102, "Dan", "A", 101], [103, "James", "A", 101], [104, "Amy", "A", 101], [105, "Anne", "A", 101], [106, "Ron", "B", 101]]}}
pd.DataFrame(d['rows']['Employee'], columns=d['headers']['Employee']).to_csv("./Employee.txt", index=None)

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("570").config("pyspark.sql.shuffle.partition","4").getOrCreate()

In [12]:
e = spark.read.option("header", True).csv("./Employee.txt")
e = e.withColumn("id", e.id.cast('int')).withColumn("managerId", e.managerId.cast('int'))
e.show()

+---+-----+----------+---------+
| id| name|department|managerId|
+---+-----+----------+---------+
|101| John|         A|     null|
|102|  Dan|         A|      101|
|103|James|         A|      101|
|104|  Amy|         A|      101|
|105| Anne|         A|      101|
|106|  Ron|         B|      101|
+---+-----+----------+---------+



Write an SQL query to report the managers with at least five direct reports.

Return the result table in any order.

The query result format is in the following example.

In [13]:
def addAlias(df, n):
    for i in df.columns:
        df=df.withColumnRenamed(i, f'{i}_{n}')
    return df

e1 = addAlias(e,1)

In [15]:
e.join(e1, e.id == e1.managerId_1, "leftouter").select("id","name", "id_1").groupBy("id","name").count().withColumnRenamed("count", "cnt").where("cnt>=5").show()

+---+----+---+
| id|name|cnt|
+---+----+---+
|101|John|  5|
+---+----+---+

